In [1]:

import nltk
from nltk import FreqDist
from nltk.tokenize import word_tokenize

# 示例文本
text = "This is a sample text to demonstrate how to calculate word familiarity using NLTK. NLTK is a powerful natural language processing library."

# 分词
words = word_tokenize(text)

# 计算词频
freq_dist = FreqDist(words)

# 输出前10个最常见的单词及它们的频率
print(freq_dist.most_common(10))

[('is', 2), ('a', 2), ('to', 2), ('NLTK', 2), ('.', 2), ('This', 1), ('sample', 1), ('text', 1), ('demonstrate', 1), ('how', 1)]


In [5]:
import nltk
from nltk import FreqDist

def familliarity_score(word):
    word = [word]
    score = FreqDist(word)
    print(score)
    return score[0]

print(familliarity_score('word'))
print(familliarity_score('word;'))

<FreqDist with 1 samples and 1 outcomes>
0
<FreqDist with 1 samples and 1 outcomes>
0


In [11]:
import nltk

# 下载WordNet
# nltk.download('wordnet')


# 访问WordNet
from nltk.corpus import wordnet
synonyms = wordnet.synsets('car')
synonyms

[Synset('car.n.01'),
 Synset('car.n.02'),
 Synset('car.n.03'),
 Synset('car.n.04'),
 Synset('cable_car.n.01')]

In [16]:
import nltk
from nltk.corpus import brown

# 下载Brown语料库（如果尚未下载）
# nltk.download('brown')

# 获取Brown语料库中的所有单词
brown_words = brown.words()

# 要查询的词
target_word = 'apple1'

# 计算目标词在Brown语料库中的出现频率
word_frequency = brown_words.count(target_word)

print(word_frequency)

# print(f"The word '{target_word}' appears {word_frequency} times in the Brown corpus.")


0


In [8]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from transformers import BertModel
import random

class MultiModalModelWithAttention(nn.Module):
    def __init__(self, hidden_dim, output_dim, show_dimension=False):
        super(MultiModalModelWithAttention, self).__init__()
        self.bert = BertModel.from_pretrained("/root/aproj/models/vbert/")
        embedding_dim = self.bert.config.hidden_size
        
        self.text_lstm = nn.LSTM(embedding_dim + 3, hidden_dim, batch_first=True)
        self.eye_tracking_lstm = nn.LSTM(3, hidden_dim, batch_first=True)
        
        # 添加注意力层
        self.attention = nn.Linear(hidden_dim * 2, 1)
        
        # 调整全连接层的输入维度，因为我们将融合来自两个LSTM的特征
        self.fc = nn.Linear(hidden_dim * 2, output_dim)

        self.show_dimension = show_dimension

        self.attentionAlignmentLayer = AttentionAlignment(128, 128, 64)
        

    def forward(self, input_ids, attention_mask, word_features, eye_tracking_features):

        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        text_embeddings = outputs.last_hidden_state
        text_features = torch.cat((text_embeddings, word_features), dim=-1)
        
        text_out, _ = self.text_lstm(text_features)
        eye_tracking_out, _ = self.eye_tracking_lstm(eye_tracking_features)
    
        if self.show_dimension:
            print(f"text_out:{text_out.shape}")
            print(f"eye_tracking_out:{eye_tracking_out.shape}")
            
        output = self.attentionAlignmentLayer(text_out, eye_tracking_out)
        if self.show_dimension:
            print(f"output.shape:{output.shape}")
        return output

class AttentionAlignment(nn.Module):
    def __init__(self, word_dim, fixation_dim, hidden_dim):
        super(AttentionAlignment, self).__init__()
        self.word_dim = word_dim
        self.fixation_dim = fixation_dim
        self.hidden_dim = hidden_dim
        
        # 添加线性层调整fixation_seq的维度
        print(f"fixation_dim:{fixation_dim}")
        print(f"word_dim:{word_dim}")
        self.linear_fixation = nn.Linear(fixation_dim, word_dim)
        # 定义注意力权重计算层
        self.attention = nn.MultiheadAttention(embed_dim=word_dim, num_heads=1)
        # 定义全连接层
        self.fc = nn.Linear(word_dim * 2, hidden_dim)
        # 定义输出层
        self.output_layer = nn.Linear(hidden_dim, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, word_seq, fixation_seq):
        # 调整fixation_seq的维度
        adjusted_fixation_seq = self.linear_fixation(fixation_seq)
        # 计算注意力权重
        attention_weights, _ = self.attention(word_seq.permute(1, 0, 2), adjusted_fixation_seq.permute(1, 0, 2), adjusted_fixation_seq.permute(1, 0, 2))
        # print(attention_weights.shape)
        # 对齐word序列和fixation序列
        aligned_word = torch.cat((word_seq, attention_weights.permute(1, 0, 2)), dim=2)
        # 使用全连接层进行特征融合
        fused_features = torch.relu(self.fc(aligned_word))      
        # 输出层
        output = self.sigmoid(self.output_layer(fused_features))
        return output

In [9]:
# Define shapes for synthetic data
batch_size = 1
max_seq_length = 512
# embedding_dim = 768  # Assuming BERT output size
hidden_dim = 64
output_dim = 1  # Binary classification

# Generate synthetic data
input_ids = torch.randint(0, 1000, (batch_size, max_seq_length))  # Example input for BERT
attention_mask = torch.ones((batch_size, max_seq_length))  # Example attention mask
word_features = torch.randn((batch_size, max_seq_length, 3))  # Example word features
eye_tracking_features = torch.randn((batch_size, max_seq_length, 3))  # Example eye-tracking features

# Initialize model
model = MultiModalModelWithAttention(hidden_dim, output_dim, show_dimension=True)

# Pass synthetic data through the model
output = model(input_ids, attention_mask, word_features, eye_tracking_features)

# Print the dimensions of the output
print(f"Final output shape: {output.shape}")

fixation_dim:128
word_dim:128
text_out:torch.Size([1, 512, 64])
eye_tracking_out:torch.Size([1, 512, 64])


RuntimeError: mat1 and mat2 shapes cannot be multiplied (512x64 and 128x128)